In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle

In [ ]:
def open_df():
    
    mapping = {
        'No Limit NG' : 'pgp_default_v1_20200420_110456.csv',
        '10% NG' : 'pgp_default_v2_20200420_111257.csv',
        '5% NG' : 'pgp_default_v3_20200420_113319.csv',
        '1% NG' : 'pgp_default_v4_20200420_114144.csv',
        '0% NG': 'pgp_default_v5_20200420_124759.csv', #'pgp_default_v0_20200420_110741.csv',
    }
    

    cnt = 0
    names = []
    for k, v in mapping.items():
        names.append(k)
        
        if cnt == 0:
            master = pd.read_csv(v, index_col='case name').T
            #, usecols = lambda column : column not in 
            #        ["norm. demand to 1","demand file","solar file" , "wind file", "problem status"]
        else:
            df = pd.read_csv(v, index_col='case name').T
            master = master.append(df)
        cnt += 1

    master['name'] = names
    master = master.set_index('name')
    
    cols_to_skip = ["norm. demand to 1","demand file","solar file" , "wind file", "problem status"]
    conv_map = {}
    for col in master.columns:
        if col in cols_to_skip:
            continue #this should stay a string
        conv_map[col] = 'float'
    
    master = master.astype(conv_map, errors='ignore')
    print(master.dtypes)
    return master
    
df = open_df()

In [ ]:
def plot_system_cost(df):
    
    fix, ax = plt.subplots()
    
    width = 0.35       # the width of the bars: can also be len(x) sequence
    
    mapping = {
        'system cost ($ or $/kWh)' : 'system cost ($/kWh)',   
    }

    for k, v in mapping.items():
        p1 = ax.bar(df.index, df[k], width, label=v)
    
    ax.set_ylim(0, ax.get_ylim()[1])
    ax.set_ylabel('system cost ($/kWh)')
    
    plt.legend()
    plt.savefig('syst_cost.png')

def plot_capacities(df):
    
    fix, ax = plt.subplots()
    
    width = 0.35       # the width of the bars: can also be len(x) sequence
    
    mapping = {
        'capacity natgas (kW)' : 'natgas (kW)',
        'capacity solar (kW)' : 'solar (kW)',
        'capacity wind (kW)' : 'wind (kW)',
        'capacity storage (kW)' : 'storage (kW)',
        'capacity from pgp storage (kW)' : 'from pgp storage (kW)',
        #'capacity pgp storage (kW)' : 'capacity pgp storage (kW)',
        'capacity to pgp storage (kW)' : 'to pgp storage (kW)',
    }

    bottoms = np.zeros(len(df.index))
    for k, v in mapping.items():
        p1 = ax.bar(df.index, df[k], width, bottom=bottoms, label=v)
        bottoms += df[k]
    
    ax.set_ylim(0, ax.get_ylim()[1]*1.5)
    ax.set_ylabel('capacity (kW)')
    
    
    plt.legend(ncol=2)
    plt.savefig('capacities.png')
    
def plot_dispatch(df):
    
    fix, ax = plt.subplots()
    
    width = 0.35       # the width of the bars: can also be len(x) sequence
    
    mapping = {
        'dispatch natgas (kW)' : 'natgas (kW)',
        'dispatch solar (kW)' : 'solar (kW)',
        'dispatch wind (kW)' : 'wind (kW)',
        'dispatch to storage (kW)' : 'to batt storage (kW)',
        'dispatch from pgp storage (kW)' : 'from pgp storage (kW)',
        #'dispatch pgp storage (kW)' : 'dispatch pgp storage (kW)',
        'dispatch to pgp storage (kW)' : 'to pgp storage (kW)',
    }

    bottoms = np.zeros(len(df.index))
    for k, v in mapping.items():
        p1 = ax.bar(df.index, df[k], width, bottom=bottoms, label=v)
        bottoms += df[k]
    
    ax.set_ylim(0, ax.get_ylim()[1]*1.5)
    ax.set_ylabel('dispatch (kW)')
    
    plt.legend(ncol=2)
    plt.savefig('dispatch.png')
    

plot_system_cost(df)
plot_capacities(df)
plot_dispatch(df)

In [ ]:
def get_pickle(f_name):
    
    pickle_in = open(f_name,'rb')
    
    return pickle.load(pickle_in)



# COPIED FROM JACKIE
###=========================================
#System Cost Calculations
##=========================================
##===========================================
def get_cost_contributions(base):
    info = base[0]
    inputs = base[1]
    results = base[2]
    # Costs over time
    wind_t = (np.multiply(inputs["FIXED_COST_WIND"], results["CAPACITY_WIND"]) +
             np.multiply(inputs["VAR_COST_WIND"], results["DISPATCH_WIND"]) )

    solar_t = (np.multiply(inputs["FIXED_COST_SOLAR"], results["CAPACITY_SOLAR"]) +
             np.multiply(inputs["VAR_COST_SOLAR"], results["DISPATCH_SOLAR"]) )

    pgp_t = (np.multiply(inputs["FIXED_COST_PGP_STORAGE"], results["CAPACITY_PGP_STORAGE"]) +
             np.multiply(inputs["FIXED_COST_TO_PGP_STORAGE"], results["CAPACITY_TO_PGP_STORAGE"]) +
             np.multiply(inputs["FIXED_COST_FROM_PGP_STORAGE"], results["CAPACITY_FROM_PGP_STORAGE"]) +
             np.multiply(inputs["VAR_COST_TO_PGP_STORAGE"], results["DISPATCH_TO_PGP_STORAGE"]) +
             np.multiply(inputs["VAR_COST_FROM_PGP_STORAGE"], results["DISPATCH_FROM_PGP_STORAGE"]) )

    batt_t = (np.multiply(inputs["FIXED_COST_STORAGE"], results["CAPACITY_STORAGE"]) +
             np.multiply(inputs["VAR_COST_TO_STORAGE"], results["DISPATCH_TO_STORAGE"]) +
             np.multiply(inputs["VAR_COST_FROM_STORAGE"], results["DISPATCH_FROM_STORAGE"]) )

    # Edited from Jackie's to remove "_CCS" as I'm just doing NATGAS
    ccs_t = (np.multiply(inputs["FIXED_COST_NATGAS"], results["CAPACITY_NATGAS"]) +
             np.multiply(inputs["VAR_COST_NATGAS"], results["DISPATCH_NATGAS"]) )


    # Mean costs
    wind_m = np.mean(wind_t)
    solar_m = np.mean(solar_t)
    pgp_m = np.mean(pgp_t)
    batt_m = np.mean(batt_t)
    ccs_m = np.mean(ccs_t)
    syst_cost = results["SYSTEM_COST"]

#    print('System Cost Contributions =')
#    print(results['SYSTEM_COST'])
#    print('My calcs =')
#    calc_sys_cost = wind_m + solar_m + pgp_m + batt_m
#    print(calc_sys_cost)
    return [wind_m, solar_m , pgp_m, batt_m, ccs_m, syst_cost]

def pkl_results():
    
    mapping = {
        'No Limit\nNatgas' : 'pgp_default_v1_SolarWind_PGPbatt.pickle',
        '10%\nNatgas' : 'pgp_default_v2_SolarWind_PGPbatt.pickle',
        '5%\nNatgas' : 'pgp_default_v3_SolarWind_PGPbatt.pickle',
        '1%\nNatgas' : 'pgp_default_v4_SolarWind_PGPbatt.pickle',
        '0%\nNatgas': 'pgp_default_v5_SolarWind_PGPbatt.pickle', #'pgp_default_v0_20200420_110741.csv',
    }

    wind_v, solar_v , pgp_v, batt_v, ccs_v, syst_cost_v, names = [], [], [], [], [], [], []
    for k, v in mapping.items():
        
        names.append(k)
        pkl = get_pickle(v)
        wind_m, solar_m , pgp_m, batt_m, ccs_m, syst_cost = get_cost_contributions(pkl)
        wind_v.append(wind_m)
        solar_v.append(solar_m)
        pgp_v.append(pgp_m)
        batt_v.append(batt_m)
        ccs_v.append(ccs_m)
        syst_cost_v.append(syst_cost)
        
    df = pd.DataFrame({
        'Names' : names,
        'Wind' : wind_v,
        'Solar' : solar_v,
        'PGP' : pgp_v,
        'Battery' : batt_v,
        'Natgas' : ccs_v,
        'System Cost' : syst_cost_v,
    })
        
    print(df.head())
    return df



def plot_split_cost():
    
    df = pkl_results()
    
    fix, ax = plt.subplots()
    
    width = 0.35       # the width of the bars: can also be len(x) sequence
    
    color_map = {
        'Wind': 'blue',
        'Solar': 'orange',
        'PGP': 'pink',
        'Battery': 'purple',
        'Natgas': 'brown',
        #Can we make Natgas “brown” and wind/solar/pgp/batt “blue”, 
        #“orange”, “pink”, “purple” in matplotlib in your stacked system cost figure?
    }

    bottoms = np.zeros(len(df.index))

    order = ['Natgas', 'Solar', 'Wind', 'Battery', 'PGP', ]#'System Cost']
    for col in order:
        
        if col == 'Names':
            continue
        vect = []
        
        if col == 'System Cost':
            p1 = ax.plot(df['Names'], df[col], marker='v', color='black', linewidth=0, label=col)
        else:
            p1 = ax.bar(df['Names'], df[col], width, bottom=bottoms, label=col, color=color_map[col])
            bottoms += df[col]
    
    ax.set_ylim(0, ax.get_ylim()[1]*1.5)
    ax.set_ylabel('System cost ($/kWh)')
    
    plt.legend(ncol=2)
    plt.savefig('split_syst_cost.png')
    plt.savefig('split_syst_cost.pdf')

plot_split_cost()